In [37]:
import nltk
import copy
import json

In [38]:
characters = [] #To be displayed as nodes;
connections = [] #To be displayed as links;
steps = 5 #The unit of the query is every 5 paragraphs;

In [39]:
#Load the fiction file
with open("~/fictions/gatsby/gatsby.txt") as file:
    paragraph_list = [line.strip() for line in file if line.strip() != ""]

In [40]:
def find_character(name, characters):
    for c in characters:
        if c['name'] == name:
            return c
    return None

def analyze_characters(paragraph, characters, connections):
    characters_in_chapter = []
    #Use NLTK to split the paragraph into sentences
    for sentence in nltk.sent_tokenize(paragraph):
        #Use NLTK to split the sentences into word tokens
        tokens = nltk.word_tokenize(sentence)
        tags = nltk.pos_tag(tokens)
        for chunk in nltk.ne_chunk(tags):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                #To merge the separated token, e.g. 'Tom' + 'Buchanan' = 'Tom Buchananxs'
                name = ' '.join(leave[0] for leave in chunk.leaves())
                #Accumulate the appearnce number of the character
                count_appearance(name, characters)
                #Get all characters who appear in the paragraph
                characters_in_chapter.append(name)
    #Eliminate duplicated names
    characters_in_chapter = list(set(characters_in_chapter))
    #Accumulate the connection number of every character in the paragraph
    count_connections(characters_in_chapter, connections)

def count_appearance(name, characters):
    character = find_character(name, characters)
    #Exist: character.count+1; Not exist: characters.append(character)
    if character:
        character['count']+=1
    else:
        character = characters.append({'name':name,'count':1})
        
def count_connections(characters_in_chapter, connections):
    for name in characters_in_chapter:
        #Softcopy the character list to store other characters
        others = copy.copy(characters_in_chapter)
        #Remove the iterator character
        others.remove(name)
        for other in others:
            #Query connections between two characters
            connection = find_connection(name, other, connections)
            #Exist: connection.count+1; Not exist: connections.append(connection)
            if connection:
                connection["count"]+=1
            else:
                connections.append({'source':name,'target':other,'count':1})
        characters_in_chapter.remove(name)
       
def find_connection(name1, name2, connections):
    for connection in connections:
        if connection["source"] == name1 and connection["target"] == name2:
            return connection
        elif connection["source"] == name2 and connection["target"] == name1:
            return connection
    return None
    
def analyze_paragraphs(paragraph_list, characters, connections, steps):
    #Merge the paragraphs according to step value
    #by default, every 5 paragraphs will be merged as one
    new_paragraph_list =  ['\n'.join(paragraph_list[x:x+steps]) for x in range(0, len(paragraph_list),steps)]
    for paragraph in new_paragraph_list:
        analyze_characters(paragraph, characters, connections)
    


In [41]:
#Generate the html and JavaScript
def generate_js(characters, connections):
    #Convert character and connections into JSON
    data='var nodes='+json.dumps(characters) + ';var links='+json.dumps(connections)+';'
    #html, css and compressed JavaScript
    head='<!DOCTYPE html><meta charset="utf-8"><style>body{padding:0;margin: 0}line{stroke-opacity:0.6;pointer-events: none;}circle{stroke: #fff;stroke-width:1.5px;}text{font-family:sans-serif;font-size:9px;pointer-events:none;cursor: default;}body{overflow:hidden;}svg{width:100vw;height:100vh;}</style><body><svg></svg></body><script src="https://d3js.org/d3.v4.min.js"></script><script>'
    content='var defaultLinkColor="#AAA",highlightLinkColor="#ff0000",defaultTextColor="#000",highlightNodeColor="#ff0000",defaultNodeColor="#308CC9",highlightTextColor="#ff0000",svg=(defaultTextColor="#000",d3.select("svg")),width=+window.innerWidth,height=+window.innerHeight,defaultTransform=d3.zoomIdentity.scale(1),color=d3.scaleOrdinal(d3.schemeCategory20),simulation=d3.forceSimulation().force("link",d3.forceLink().id(function(e){return e.name})).force("charge",d3.forceManyBody()).force("center",d3.forceCenter(width/2,height/2)),g=svg.append("g").attr("id","group"),linkElements=g.selectAll("line").data(links).enter().append("line").attr("stroke",defaultLinkColor).attr("stroke-width",function(e){return Math.sqrt(e.count)*1.5}),nodeElements=g.selectAll("circle").data(nodes).enter().append("circle").attr("r",5).attr("fill",defaultNodeColor).call(d3.drag().on("start",dragstarted).on("drag",dragged).on("end",dragended)).on("mouseover",function(e){selectNode(e)}).on("click",function(e){selectNode(e)}).on("mouseleave",function(e){unselectNode()}),textElements=g.selectAll("text").data(nodes).enter().append("text").text(function(e){return e.name}).attr("x",6).attr("y",3).attr("fill",defaultTextColor);function ticked(){nodeElements.attr("cx",function(e){return e.x}).attr("cy",function(e){return e.y}),textElements.attr("x",function(e){return e.x+7}).attr("y",function(e){return e.y+3}),linkElements.attr("x1",function(e){return e.source.x}).attr("y1",function(e){return e.source.y}).attr("x2",function(e){return e.target.x}).attr("y2",function(e){return e.target.y})}function dragstarted(e){d3.event.active||simulation.alphaTarget(.3).restart(),e.fx=e.x,e.fy=e.y}function dragged(e){e.fx=d3.event.x,e.fy=d3.event.y}function dragended(e){d3.event.active||simulation.alphaTarget(0),e.fx=null,e.fy=null}simulation.nodes(nodes).on("tick",ticked),simulation.force("link").links(links);var zoom=d3.zoom().scaleExtent([.1,10]).on("zoom",zoomed).filter(function(){return 0===event.button||1===event.button});function zoomed(){g.attr("transform",d3.event.transform)}function getNeighbors(n){return links.reduce(function(e,t){return t.target.name===n.name?e[t.source.name]=t.count:t.source.name===n.name&&(e[t.target.name]=t.count),e},[n.name])}function moveToFront(e){return document.getElementById("group").appendChild(e)}function isNeighborLink(e,t){return t.target.name===e.name||t.source.name===e.name}function isNeighborNode(e,t){return null!=t&&null!=t[e.name]}function getLinkColor(e,t){return isNeighborLink(e,t)?highlightLinkColor:defaultLinkColor}function getNodeColor(e,t){return isNeighborNode(e,t)?highlightNodeColor:defaultNodeColor}function getTextColor(e,t){return isNeighborNode(e,t)?highlightTextColor:defaultTextColor}function selectNode(t,e){var n=getNeighbors(t);linkElements.attr("stroke",function(e){return getLinkColor(t,e)}),nodeElements.attr("fill",function(e){return e.name==t.name?highlightNodeColor:getNodeColor(e,n)}),textElements.attr("fill",function(e){return e.name==t.name?highlightTextColor:getTextColor(e,n)}),nodeElements.attr("r",function(e){return e.name==t.name?7:5}),linkElements.select(function(e){isNeighborLink(t,e)&&moveToFront(this)}),nodeElements.select(function(e){isNeighborNode(e,n)&&moveToFront(this)}),textElements.select(function(e){isNeighborNode(e,n)&&moveToFront(this)}),nodeElements.select(function(e){e.name==t.name&&moveToFront(this)}),textElements.select(function(e){e.name==t.name&&moveToFront(this)})}function unselectNode(e){linkElements.attr("stroke",defaultLinkColor),nodeElements.attr("fill",defaultNodeColor),textElements.attr("fill",defaultTextColor),nodeElements.attr("r",5)}svg.call(zoom).call(zoom.transform,defaultTransform);</script>'
    return head+data+content

In [42]:
#write into html file
analyze_paragraphs(paragraph_list, characters, connections, steps)
js = generate_js(characters, connections)
with open("diagram_1.html", "w", encoding="utf-8") as text_file:
    text_file.write(js)